# Classifying patches

Training a CNN to classify patches of plastic waste

- Diving the dataset
- Creating a dataloader
- Performing transforms

## Divinding the dataset

In [1]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader

In [2]:
from torchvision import transforms as transforms

In [3]:
dataset = ImageFolder("./dataset2/", transform=Compose([ToTensor()]))

In [4]:
len(dataset)

33714

In [14]:
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=.20)
train_idx, val_idx = train_test_split(train_idx, test_size=0.20)

In [401]:
import pickle

pickle.dump([train_idx, val_idx, test_idx], open("data.pickle", "wb"))

In [15]:
train_data = Subset(dataset, train_idx)
val_data = Subset(dataset, val_idx)
test_data = Subset(dataset, test_idx)

In [16]:
BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, BATCH_SIZE, shuffle=True)

In [8]:
for i in train_dataloader:
    print(i[0].shape, i[1])
    break

torch.Size([32, 3, 64, 64]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 1])


## CNN Network

In [9]:
from torch import nn
import torch.functional as F

In [52]:
class PlasticClassifier(nn.Module):
    def __init__(self):
        super(PlasticClassifier, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(16, 8, 3, stride=2),
            nn.ReLU()
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(8*6*6, 128),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(64, 2),
        )
        
        
    def forward(self, x):
        out = self.conv_layers(x)
        out = nn.Flatten()(out)
        out = self.linear_layers(out)
        return out
    
model = PlasticClassifier()

In [85]:
8*6*6, 256/2

(288, 128.0)

In [11]:
for i in train_dataloader:
    print(model(i[0]).shape)
    break

torch.Size([32, 2])


In [53]:
optimizer = torch.optim.SGD(params=model.parameters(),
                           lr=0.05)

loss_fn = nn.CrossEntropyLoss()

In [112]:
# logging the training
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [54]:
# training
epochs = 50
train_loss1 = []
val_loss1 = []
train_acc1 = []
val_acc1 = []

for epoch in range(epochs):
    
    # training
    model.train(True)
    loss_train = 0
    train_acc = 0
    
    print(f"\n\nEpoch: {epoch}\n{'-'*10}")
    
    for batch, (data, labels) in enumerate(train_dataloader):
                
        pred = model(data)
        
        # loss
        loss = loss_fn(pred, labels)
        loss_train += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # accuracy
        train_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1) == labels).sum().item()
        
        
        if batch%50 == 0:
            print(f"\tLoss: {loss.item():>6f}, Images:{100*((batch*BATCH_SIZE)/len(train_dataloader.dataset)):.2f}%")
    
    train_acc /= len(train_dataloader.dataset)
    print(f"Epoch: {epoch:>5}, loss: {loss_train}, acc: {train_acc:>4f}")
    
    train_loss1.append(loss_train)
    train_acc1.append(train_acc)
    # writer.add_scalar("Loss/train", loss_train, epoch)
    # writer.add_scalar("Accuracy/train", train_acc, epoch)
    
    model.eval()
    
    # validation
    with torch.inference_mode():
        loss_val = 0
        test_acc = 0
        for data, labels in val_dataloader:
            pred = model(data)
            
            loss = loss_fn(pred, labels)
            loss_val += loss
            
            # accuracy
            test_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1) == labels).sum().item()
            
        test_acc /= len(test_dataloader.dataset)
        val_loss1.append(loss_val)
        val_acc1.append(test_acc)
        # writer.add_scalar("Loss/val", loss_val, epoch)
        # writer.add_scalar("Accuracy/val", train_acc, epoch)



Epoch: 0
----------
	Loss: 0.694176, Images:0.00%
	Loss: 0.689962, Images:25.08%
	Loss: 0.693932, Images:50.16%
	Loss: 0.692694, Images:75.24%
Epoch:     0, loss: 138.5995498895645, acc: 0.511912


Epoch: 1
----------
	Loss: 0.689462, Images:0.00%
	Loss: 0.686250, Images:25.08%
	Loss: 0.689572, Images:50.16%
	Loss: 0.690980, Images:75.24%
Epoch:     1, loss: 138.28362107276917, acc: 0.525235


Epoch: 2
----------
	Loss: 0.682947, Images:0.00%
	Loss: 0.634799, Images:25.08%
	Loss: 0.534720, Images:50.16%
	Loss: 0.493589, Images:75.24%
Epoch:     2, loss: 120.74755993485451, acc: 0.677900


Epoch: 3
----------
	Loss: 0.457336, Images:0.00%
	Loss: 0.550659, Images:25.08%
	Loss: 0.424312, Images:50.16%
	Loss: 0.440128, Images:75.24%
Epoch:     3, loss: 101.6241996884346, acc: 0.763323


Epoch: 4
----------
	Loss: 0.426599, Images:0.00%
	Loss: 0.581118, Images:25.08%
	Loss: 0.515029, Images:50.16%
	Loss: 0.413820, Images:75.24%
Epoch:     4, loss: 97.57088965177536, acc: 0.784796


Epoch:

In [55]:
pickle.dump({
    "train_acc": train_acc1,
    "train_loss": train_loss1,
    "val_acc": val_acc1,
    "val_loss": val_loss1
}, open("results/CNN2.pickle", "wb"))

In [150]:
torch.save(model, "models/plasticDetector.pt")

In [153]:
train_data.dataset.class_to_idx

{'not_plastic': 0, 'plastic': 1}

## Classifier 2

In [25]:
dataset = ImageFolder("./dataset5/", transform=Compose([ToTensor()]))

In [26]:
len(dataset)

9971

In [38]:
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=.20)
train_idx, val_idx = train_test_split(train_idx, test_size=0.20)

In [39]:
train_data = Subset(dataset, train_idx)
val_data = Subset(dataset, val_idx)
test_data = Subset(dataset, test_idx)

In [40]:
BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, BATCH_SIZE, shuffle=True)

In [300]:
class PlasticClassifier2(nn.Module):
    def __init__(self):
        super(PlasticClassifier2, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 5, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.Conv2d(64, 86, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(86, 128, 3),
            nn.ReLU(),
            nn.Conv2d(128, 64, 1),
            nn.ReLU()
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(64*10*10, 1024),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
        
        
        
    def forward(self, x):
        out = self.conv_layers(x)
        out = nn.Flatten()(out)
        out = self.linear_layers(out)
        return out
    
model2 = PlasticClassifier2()

In [49]:
class PlasticClassifier3(nn.Module):
    def __init__(self):
        super(PlasticClassifier3, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 128, 3),
            nn.ReLU(),
            nn.Conv2d(128, 64, 5),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3),            
            nn.ReLU(),
            nn.Conv2d(32, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 8, 3),
            nn.ReLU()
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(8*52*52, 1024),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
        
    def forward(self, x):
        out = self.conv_layers(x)
        out = nn.Flatten()(out)
        out = self.linear_layers(out)
        return out
    
model3 = PlasticClassifier3()

In [61]:
8*52*52

21632

In [360]:
class PlasticClassifier4(nn.Module):
    def __init__(self):
        super(PlasticClassifier4, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3),
            nn.Dropout(),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(16, 8, 3, stride=2),
            nn.ReLU()
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(8*6*6, 128),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(64, 2),
        )
        
        
    def forward(self, x):
        out = self.conv_layers(x)
        out = nn.Flatten()(out)
        out = self.linear_layers(out)
        return out
    
model4 = PlasticClassifier4()

In [32]:
data_transforms = Compose([
    transforms.RandomAffine(
        degrees=(10, 70)),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomApply([
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip()],p=0.6),
    transforms.Resize((64,64), antialias=None)
])

In [33]:
for i in train_dataloader:
    data_transforms(i[0])
    print(model3(i[0]).shape)
    break

torch.Size([32, 2])


In [289]:
train_data.dataset.class_to_idx

{'not_plastic': 0, 'plastic': 1}

In [50]:
optimizer = torch.optim.SGD(params=model3.parameters(),
                           lr=0.1,)
                           # weight_decay=0.0001)

loss_fn = nn.CrossEntropyLoss()

In [35]:
from random import random

In [36]:
random()

0.5859825038422828

In [51]:
# training
epochs = 30
model_t = model3
train_acc2 = []
train_loss2 = []
val_acc2 = []
val_loss2 = []


for epoch in range(epochs):
    
    # training
    model_t.train(True)
    loss_train = 0
    train_acc = 0
    
    print(f"\n\nEpoch: {epoch}\n{'-'*10}")
    
    for batch, (data, labels) in enumerate(train_dataloader):
        
        if random() > 0.7:
            data = data_transforms(data)
        
        pred = model_t(data)
        
        # loss
        loss = loss_fn(pred, labels)
        loss_train += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # accuracy
        train_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1) == labels).sum().item()
        
        
        if batch%50 == 0:
            print(f"\tLoss: {loss.item():>6f}, Images:{100*((batch*BATCH_SIZE)/len(train_dataloader.dataset)):.2f}%")
    
    train_acc /= len(train_dataloader.dataset)
    print(f"Epoch: {epoch:>5}, loss: {loss_train}, acc: {train_acc:>4f}")
    train_loss2.append(loss_train)
    train_acc2.append(train_acc)
    # writer.add_scalar("Loss/train", loss_train, epoch)
    # writer.add_scalar("Accuracy/train", train_acc, epoch)
    
    model_t.eval()
    
    # validation
    with torch.inference_mode():
        loss_val = 0
        test_acc = 0
        for data, labels in val_dataloader:
            pred = model_t(data)
            
            loss = loss_fn(pred, labels)
            loss_val += loss
            
            # accuracy
            test_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1) == labels).sum().item()
            
        test_acc /= len(test_dataloader.dataset)
        val_loss2.append(loss_val)
        val_acc2.append(test_acc)
        # writer.add_scalar("Loss/val", loss_val, epoch)
        # writer.add_scalar("Accuracy/val", train_acc, epoch)



Epoch: 0
----------
	Loss: 0.690196, Images:0.00%
	Loss: 0.695276, Images:25.08%
	Loss: 0.696796, Images:50.16%
	Loss: 0.699331, Images:75.24%
Epoch:     0, loss: 138.60688894987106, acc: 0.512226


Epoch: 1
----------
	Loss: 0.690005, Images:0.00%
	Loss: 0.696243, Images:25.08%
	Loss: 0.691450, Images:50.16%
	Loss: 0.698618, Images:75.24%
Epoch:     1, loss: 138.6279951930046, acc: 0.503292


Epoch: 2
----------
	Loss: 0.673465, Images:0.00%
	Loss: 0.721081, Images:25.08%
	Loss: 0.682617, Images:50.16%
	Loss: 0.483316, Images:75.24%
Epoch:     2, loss: 135.37070295214653, acc: 0.581505


Epoch: 3
----------
	Loss: 0.890743, Images:0.00%
	Loss: 0.583149, Images:25.08%
	Loss: 0.674459, Images:50.16%
	Loss: 0.558115, Images:75.24%
Epoch:     3, loss: 125.79009971022606, acc: 0.659404


Epoch: 4
----------
	Loss: 0.654522, Images:0.00%
	Loss: 0.697204, Images:25.08%
	Loss: 0.692977, Images:50.16%
	Loss: 0.698317, Images:75.24%
Epoch:     4, loss: 136.2929750084877, acc: 0.526489


Epoch

KeyboardInterrupt: 

Model 4:
- just bags and bottles vs everything else

Model 5:
- increased padding to 10px to capture waters as well, and dropout in conv layers

Model 6:
- deeper model3 trained on padded set

In [389]:
writer.close()

In [397]:
torch.save(model3, "models/plasticDetector7.pt")

https://medium.com/@Sanskriti.Singh/an-emphasis-on-the-minimization-of-false-negatives-false-positives-in-binary-classification-9c22f3f9f73

# calc

In [60]:
a=np.array([3,2,3,4,3,5])
a/a.sum() * 750

array([112.5,  75. , 112.5, 150. , 112.5, 187.5])